In [14]:
%autosave 60

Autosaving every 60 seconds


## 1. Setup

In [1]:
import tensorflow as tf
import keras
import numpy as np
print(tf.__version__)

2.4.1


## 2. Model function

In [6]:
def get_model():
    inputs = keras.Input(shape=(32,))
    outputs = keras.layers.Dense(1)(inputs)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer='adam',loss='mean_squared_error')
    return model

In [7]:
model = get_model()
test_input = np.random.random((128, 32))
test_target = np.random.random((128,1))
model.fit(test_input, test_target)

4/4 [==============================] - 0s 787us/step - loss: 0.4514


## 3. Saving H5 format
Note : Keras also supports saving a single HDF5 file containing the model's architecture, weights values, and compile() information. It is a light-weight alternative to SavedModel.



In [8]:
model.save("Ex_h5_model.h5")

In [9]:
reconstructed_model = keras.models.load_model("Ex_h5_model.h5")

In [10]:
np.testing.assert_allclose(model.predict(test_input), reconstructed_model.predict(test_input))

In [13]:
#reconstructed_model.predict(test_input)

### Note
 The reconstructed model is already compiled and has retained the optimizer
 state, so training can resume:


In [11]:
reconstructed_model.fit(test_input, test_target)

4/4 [==============================] - 0s 993us/step - loss: 0.3996


## 4. Comment
### Limitations
Compared to the SavedModel format, there are two things that don't get included in the H5 file:

1. **External losses & metrics** added via model.add_loss() & model.add_metric() are not saved (unlike SavedModel). If you have such losses & metrics on your model and you want to resume training, you need to add these losses back yourself after loading the model. Note that this does not apply to losses/metrics created inside layers via self.add_loss() & self.add_metric(). As long as the layer gets loaded, these losses & metrics are kept, since they are part of the call method of the layer.
2. **The computation graph of custom objects** such as custom layers is not included in the saved file. At loading time, Keras will need access to the Python classes/functions of these objects in order to reconstruct the model. See Custom objects.
